In [2]:
# --- Establecer directorio de trabajo raíz ---
import os
os.chdir("/workspaces/fruit-detector-pricer")
print("Directorio de trabajo actual:", os.getcwd())

# --- Importar librerías necesarias ---
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image

# --- Rutas base del dataset ---
input_base = "./ProyectoFinal/Dataset_Filtrado"
output_base = "./ProyectoFinal/Dataset_Segmentado"

# Crear carpeta de salida si no existe
os.makedirs(output_base, exist_ok=True)

# --- Función de segmentación Otsu (forzar fruta blanca / fondo negro) ---
def segmentar_otsu(imagen_path):
    """
    Segmenta la fruta usando Otsu. La fruta quedará blanca (255) y el fondo negro (0).
    """
    img = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    # Suavizar para reducir ruido
    blur = cv2.GaussianBlur(img, (5, 5), 0)

    # Umbralización de Otsu (en modo binario inverso)
    # Esto suele dejar los objetos oscuros (fruta) blancos y fondo negro
    _, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Conectividad: mantener la componente más grande (fruta)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num_labels > 1:
        largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
        mask = np.where(labels == largest, 255, 0).astype("uint8")

    # Operaciones morfológicas para limpiar bordes y huecos
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

    return mask

# --- Procesar imágenes de train y test ---
for folder in ["train", "test"]:
    input_dir = os.path.join(input_base, folder)
    output_dir = os.path.join(output_base, folder)
    os.makedirs(output_dir, exist_ok=True)

    for clase in tqdm(os.listdir(input_dir), desc=f"Procesando {folder}"):
        clase_in = os.path.join(input_dir, clase)
        clase_out = os.path.join(output_dir, clase)
        os.makedirs(clase_out, exist_ok=True)

        for img_name in os.listdir(clase_in):
            path_in = os.path.join(clase_in, img_name)
            mask = segmentar_otsu(path_in)
            if mask is not None:
                out_path = os.path.join(clase_out, img_name)
                Image.fromarray(mask).save(out_path)

print("✅ Segmentación completada: fruta blanca, fondo negro en train y test.")


Directorio de trabajo actual: /workspaces/fruit-detector-pricer


Procesando train:   0%|          | 0/10 [00:00<?, ?it/s]

Procesando test: 100%|██████████| 10/10 [00:02<00:00,  4.22it/s]

✅ Segmentación completada: fruta blanca, fondo negro en train y test.
